## Import libraries

In [30]:
import io
import itertools

import numpy as np
import sklearn.metrics

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

import matplotlib.pyplot as plt

## Load and preprocess data

In [31]:
train_data = np.load('dataset/Trousers & Jeans - Male - train.npz')
validation_data = np.load('dataset/Trousers & Jeans - Male - validation.npz')
test_data = np.load('dataset/Trousers & Jeans - Male - test.npz')

In [32]:
np.unique(train_data)

array(['images', 'labels'], dtype='<U6')

In [33]:
images_train, labels_train = train_data['images'], train_data['labels']
images_val, labels_val = validation_data['images'], validation_data['labels']
images_test, labels_test = test_data['images'], test_data['labels']

In [34]:
np.unique(labels_train)

array([0, 1])

In [35]:
images_train.shape

(2002, 120, 90, 3)

In [36]:
labels_train.shape

(2002,)

In [37]:
images_validation.shape

(250, 120, 90, 3)

In [38]:
labels_validation.shape

(250,)

In [39]:
images_test.shape

(250, 120, 90, 3)

In [40]:
labels_test.shape

(250,)

In [41]:
# standardize 
images_train = images_train/255.0
images_val = images_val/255.0
images_test = images_test/255.0

## Create model

### Define some hyperparameters

In [42]:
EPOCHS = 15
BATCH_SIZE = 64

In [43]:
HP_KERNEL_SIZE_1 = hp.HParam('kernel size 1', hp.Discrete([3,5]))
HP_KERNEL_SIZE_2 = hp.HParam('kernel size 2', hp.Discrete([3,5]))
HP_KERNEL_NUM = hp.HParam('kernel num', hp.Discrete([32,64]))
HP_DENSE_SIZE = hp.HParam('dense size', hp.Discrete([256,512]))

METRIC = 'accuracy'

with tf.summary.create_file_writer(r'Logs/Model 1 (Trousers&Jeans_Male)/hparam_tuning').as_default():
    hp.hparams_config(
        hparams = [HP_KERNEL_SIZE_1,HP_KERNEL_SIZE_2,HP_KERNEL_NUM,HP_DENSE_SIZE],
        metrics = [hp.Metric(METRIC, display_name = 'accuracy')]
    )

### Model architecture

In [44]:
def train_test_model(hparams, session_num):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(hparams[HP_KERNEL_NUM], hparams[HP_KERNEL_SIZE_1], activation = 'relu', input_shape = (120,90,3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(hparams[HP_KERNEL_NUM], hparams[HP_KERNEL_SIZE_2], activation = 'relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(hparams[HP_DENSE_SIZE], activation='relu'),
        tf.keras.layers.Dense(2, activation = 'softmax')
    ])
    
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
    model.compile(optimizer = 'adam', loss = loss_fn, metrics = ['accuracy'])
    
    log_dir = "Logs\\Model 1 (Trousers&Jeans_Male)\\fit\\" + f"run-{session_num}"
    
    # functions for creating confusion matrix
    def plot_confusion_matrix(cm, class_names):
        """
        Returns a matplotlib figure containing the plotted confusion matrix.

        Args:
        cm (array, shape = [n, n]): a confusion matrix of integer classes
        class_names (array, shape = [n]): String names of the integer classes
        """
        figure = plt.figure(figsize=(12, 12))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Normalize the confusion matrix.
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')

        return figure

    def plot_to_image(figure):
        """Converts the matplotlib plot specified by 'figure' to a PNG image and
        returns it. The supplied figure is closed and inaccessible after this call."""

        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format='png')

        # Closing the figure prevents it from being displayed directly inside the notebook.
        plt.close(figure)

        buf.seek(0)

        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)

        # Add the batch dimension
        image = tf.expand_dims(image, 0)

        return image

    # Define a file writer variable for logging purposes
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')

    def log_confusion_matrix(epoch, logs):
        # Use the model to predict the values from the validation dataset.
        test_pred_raw = model.predict(images_val)
        test_pred = np.argmax(test_pred_raw, axis=1)

        # Calculate the confusion matrix.
        cm = sklearn.metrics.confusion_matrix(labels_val, test_pred)

        # Log the confusion matrix as an image summary.
        figure = plot_confusion_matrix(cm, class_names=['Trousers', 'Jeans'])
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with file_writer_cm.as_default():
            tf.summary.image("Confusion Matrix", cm_image, step=epoch)
            
    # callbacks
    early_stopping = tf.keras.callbacks.EarlyStopping(patience = 2, restore_best_weights = True)
    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end = log_confusion_matrix)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir, profile_batch = 0)
    
    model.fit(images_train,
              labels_train,
              epochs = EPOCHS,
              batch_size = BATCH_SIZE,
              callbacks = [tensorboard_callback, cm_callback, early_stopping],
              validation_data = (images_val, labels_val),
              verbose = 1)
    
    _, accuracy = model.evaluate(images_val, labels_val)
    
    model.save(r'saved_models\Model 1 (Trousers&Jeans_Male)\Run-{}'.format(session_num))
    
    return accuracy

In [45]:
def run(log_dir, hparams, session_num):
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar(METRIC,accuracy,step=1)

In [46]:
session_num = 1
for kernel_size_1 in HP_KERNEL_SIZE_1.domain.values:
    for kernel_num in HP_KERNEL_NUM.domain.values:
        for kernel_size_2 in HP_KERNEL_SIZE_2.domain.values:
            for dense_size in HP_DENSE_SIZE.domain.values:
                hparams = {
                    HP_KERNEL_SIZE_1 : kernel_size_1,
                    HP_KERNEL_SIZE_2 : kernel_size_2,
                    HP_KERNEL_NUM : kernel_num,
                    HP_DENSE_SIZE : dense_size
                }
                run_name = f'run-{session_num}'
                print('--- Starting Trial:',run_name)
                print({h.name: hparams[h] for h in hparams})
                run("Logs/Model 1 (Trousers&Jeans_Male)/hparam_tuning/" + run_name, hparams, session_num)

                session_num += 1

--- Starting Trial: run-1
{'kernel size 1': 3, 'kernel size 2': 3, 'kernel num': 32, 'dense size': 256}
Epoch 1/15
32/32 [==============================] - 11s 351ms/step - loss: 1.1978 - accuracy: 0.5919 - val_loss: 0.5948 - val_accuracy: 0.6800
Epoch 2/15
32/32 [==============================] - 10s 321ms/step - loss: 0.4800 - accuracy: 0.7677 - val_loss: 0.4118 - val_accuracy: 0.8000
Epoch 3/15
32/32 [==============================] - 10s 322ms/step - loss: 0.4112 - accuracy: 0.8172 - val_loss: 0.3890 - val_accuracy: 0.8040
Epoch 4/15
32/32 [==============================] - 10s 327ms/step - loss: 0.3520 - accuracy: 0.8382 - val_loss: 0.3514 - val_accuracy: 0.8600
Epoch 5/15
32/32 [==============================] - 11s 334ms/step - loss: 0.3465 - accuracy: 0.8511 - val_loss: 0.3709 - val_accuracy: 0.8320
Epoch 6/15
8/8 [==============================] - 0s 41ms/step - loss: 0.3514 - accuracy: 0.8600
Instructions for updating:
This property should not be used in TensorFlow 2.0, as up

32/32 [==============================] - 22s 699ms/step - loss: 0.2761 - accuracy: 0.8771 - val_loss: 0.3175 - val_accuracy: 0.8800
Epoch 7/15
8/8 [==============================] - 1s 79ms/step - loss: 0.3095 - accuracy: 0.8680
INFO:tensorflow:Assets written to: saved_models\Model 1 (Trousers&Jeans_Male)\Run-6\assets
--- Starting Trial: run-7
{'kernel size 1': 3, 'kernel size 2': 5, 'kernel num': 64, 'dense size': 256}
Epoch 1/15
32/32 [==============================] - 31s 977ms/step - loss: 0.7500 - accuracy: 0.6583 - val_loss: 0.4841 - val_accuracy: 0.7600
Epoch 2/15
32/32 [==============================] - 31s 969ms/step - loss: 0.4649 - accuracy: 0.7967 - val_loss: 0.3932 - val_accuracy: 0.8160
Epoch 3/15
32/32 [==============================] - 31s 967ms/step - loss: 0.4691 - accuracy: 0.7837 - val_loss: 0.4540 - val_accuracy: 0.7880
Epoch 4/15
32/32 [==============================] - 31s 978ms/step - loss: 0.3869 - accuracy: 0.8252 - val_loss: 0.3822 - val_accuracy: 0.8320
Epoc

Epoch 4/15
32/32 [==============================] - 17s 519ms/step - loss: 0.4373 - accuracy: 0.8022 - val_loss: 0.4904 - val_accuracy: 0.7840
Epoch 5/15
32/32 [==============================] - 17s 537ms/step - loss: 0.4073 - accuracy: 0.8102 - val_loss: 0.3802 - val_accuracy: 0.7960
Epoch 6/15
32/32 [==============================] - 17s 531ms/step - loss: 0.3478 - accuracy: 0.8367 - val_loss: 0.3555 - val_accuracy: 0.8240
Epoch 7/15
32/32 [==============================] - 17s 526ms/step - loss: 0.3479 - accuracy: 0.8457 - val_loss: 0.3973 - val_accuracy: 0.8320
Epoch 8/15
8/8 [==============================] - 0s 56ms/step - loss: 0.3555 - accuracy: 0.8240
INFO:tensorflow:Assets written to: saved_models\Model 1 (Trousers&Jeans_Male)\Run-12\assets
--- Starting Trial: run-13
{'kernel size 1': 5, 'kernel size 2': 3, 'kernel num': 64, 'dense size': 256}
Epoch 1/15
32/32 [==============================] - 23s 721ms/step - loss: 1.0272 - accuracy: 0.5924 - val_loss: 0.5781 - val_accuracy